In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.13.0


In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.trainable_variables)

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def batch_norm() :
    return tf.keras.layers.BatchNormalization()

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(batch_norm())
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(batch_norm())
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_batchnorm'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.trainable_variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...


C:\Users\knh11\anaconda3\envs\tfpy3.10\lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch: [ 0] [    0/  468] time: 1.9088, train_loss: 0.37498608, train_accuracy: 0.2266, test_Accuracy: 0.2101
Epoch: [ 0] [    1/  468] time: 2.6264, train_loss: 0.37795192, train_accuracy: 0.4141, test_Accuracy: 0.3406
Epoch: [ 0] [    2/  468] time: 3.3103, train_loss: 0.28402072, train_accuracy: 0.5469, test_Accuracy: 0.4653
Epoch: [ 0] [    3/  468] time: 4.0873, train_loss: 0.26579946, train_accuracy: 0.6016, test_Accuracy: 0.5293
Epoch: [ 0] [    4/  468] time: 5.0230, train_loss: 0.30336562, train_accuracy: 0.5859, test_Accuracy: 0.5616
Epoch: [ 0] [    5/  468] time: 5.9770, train_loss: 0.22867920, train_accuracy: 0.5312, test_Accuracy: 0.5765
Epoch: [ 0] [    6/  468] time: 6.7952, train_loss: 0.31918359, train_accuracy: 0.6016, test_Accuracy: 0.5989
Epoch: [ 0] [    7/  468] time: 7.6497, train_loss: 0.22776251, train_accuracy: 0.5781, test_Accuracy: 0.6237
Epoch: [ 0] [    8/  468] time: 8.4211, train_loss: 0.24392252, train_accuracy: 0.6562, test_Accuracy: 0.6430
Epoch: [ 0

Epoch: [ 0] [   74/  468] time: 69.4632, train_loss: 0.17338468, train_accuracy: 0.9219, test_Accuracy: 0.9322
Epoch: [ 0] [   75/  468] time: 70.3409, train_loss: 0.10405365, train_accuracy: 0.9609, test_Accuracy: 0.9302
Epoch: [ 0] [   76/  468] time: 71.3062, train_loss: 0.21593376, train_accuracy: 0.9141, test_Accuracy: 0.9297
Epoch: [ 0] [   77/  468] time: 72.3201, train_loss: 0.07854304, train_accuracy: 0.9453, test_Accuracy: 0.9299
Epoch: [ 0] [   78/  468] time: 73.4504, train_loss: 0.13827366, train_accuracy: 0.9375, test_Accuracy: 0.9300
Epoch: [ 0] [   79/  468] time: 74.3431, train_loss: 0.07250805, train_accuracy: 0.9844, test_Accuracy: 0.9307
Epoch: [ 0] [   80/  468] time: 75.2998, train_loss: 0.19151333, train_accuracy: 0.9375, test_Accuracy: 0.9286
Epoch: [ 0] [   81/  468] time: 76.1325, train_loss: 0.16006896, train_accuracy: 0.9062, test_Accuracy: 0.9293
Epoch: [ 0] [   82/  468] time: 77.0046, train_loss: 0.16126987, train_accuracy: 0.9375, test_Accuracy: 0.9294
E

Epoch: [ 0] [  148/  468] time: 134.4493, train_loss: 0.11137289, train_accuracy: 0.9531, test_Accuracy: 0.9507
Epoch: [ 0] [  149/  468] time: 135.4401, train_loss: 0.08859080, train_accuracy: 0.9766, test_Accuracy: 0.9490
Epoch: [ 0] [  150/  468] time: 136.2933, train_loss: 0.05894423, train_accuracy: 0.9531, test_Accuracy: 0.9478
Epoch: [ 0] [  151/  468] time: 137.3200, train_loss: 0.03823855, train_accuracy: 0.9688, test_Accuracy: 0.9469
Epoch: [ 0] [  152/  468] time: 138.2469, train_loss: 0.07140889, train_accuracy: 0.9766, test_Accuracy: 0.9454
Epoch: [ 0] [  153/  468] time: 139.2289, train_loss: 0.11194488, train_accuracy: 0.9453, test_Accuracy: 0.9447
Epoch: [ 0] [  154/  468] time: 140.1597, train_loss: 0.15801601, train_accuracy: 0.9531, test_Accuracy: 0.9453
Epoch: [ 0] [  155/  468] time: 141.1256, train_loss: 0.06426977, train_accuracy: 0.9766, test_Accuracy: 0.9445
Epoch: [ 0] [  156/  468] time: 142.0412, train_loss: 0.05003895, train_accuracy: 0.9844, test_Accuracy:

Epoch: [ 0] [  222/  468] time: 201.5201, train_loss: 0.06784455, train_accuracy: 0.9531, test_Accuracy: 0.9500
Epoch: [ 0] [  223/  468] time: 202.4757, train_loss: 0.08945530, train_accuracy: 0.9766, test_Accuracy: 0.9479
Epoch: [ 0] [  224/  468] time: 203.4168, train_loss: 0.13973147, train_accuracy: 0.9375, test_Accuracy: 0.9503
Epoch: [ 0] [  225/  468] time: 204.2493, train_loss: 0.07665570, train_accuracy: 0.9531, test_Accuracy: 0.9520
Epoch: [ 0] [  226/  468] time: 205.1087, train_loss: 0.07766627, train_accuracy: 0.9844, test_Accuracy: 0.9526
Epoch: [ 0] [  227/  468] time: 205.9310, train_loss: 0.05872222, train_accuracy: 0.9531, test_Accuracy: 0.9547
Epoch: [ 0] [  228/  468] time: 206.7733, train_loss: 0.08210392, train_accuracy: 0.9688, test_Accuracy: 0.9552
Epoch: [ 0] [  229/  468] time: 207.6006, train_loss: 0.07959814, train_accuracy: 0.9766, test_Accuracy: 0.9561
Epoch: [ 0] [  230/  468] time: 208.5010, train_loss: 0.11047380, train_accuracy: 0.9531, test_Accuracy:

Epoch: [ 0] [  296/  468] time: 267.3281, train_loss: 0.06046615, train_accuracy: 0.9922, test_Accuracy: 0.9605
Epoch: [ 0] [  297/  468] time: 268.2622, train_loss: 0.06138577, train_accuracy: 0.9922, test_Accuracy: 0.9596
Epoch: [ 0] [  298/  468] time: 269.2846, train_loss: 0.03504294, train_accuracy: 0.9922, test_Accuracy: 0.9579
Epoch: [ 0] [  299/  468] time: 270.1752, train_loss: 0.05637325, train_accuracy: 0.9609, test_Accuracy: 0.9539
Epoch: [ 0] [  300/  468] time: 271.0141, train_loss: 0.14384969, train_accuracy: 0.9219, test_Accuracy: 0.9523
Epoch: [ 0] [  301/  468] time: 271.9642, train_loss: 0.12505911, train_accuracy: 0.9453, test_Accuracy: 0.9508
Epoch: [ 0] [  302/  468] time: 272.8582, train_loss: 0.10848748, train_accuracy: 0.9609, test_Accuracy: 0.9510
Epoch: [ 0] [  303/  468] time: 273.7826, train_loss: 0.10270480, train_accuracy: 0.9688, test_Accuracy: 0.9515
Epoch: [ 0] [  304/  468] time: 274.5439, train_loss: 0.06623847, train_accuracy: 0.9766, test_Accuracy:

Epoch: [ 0] [  370/  468] time: 332.7617, train_loss: 0.13812558, train_accuracy: 0.9453, test_Accuracy: 0.9662
Epoch: [ 0] [  371/  468] time: 333.5292, train_loss: 0.11269311, train_accuracy: 0.9688, test_Accuracy: 0.9665
Epoch: [ 0] [  372/  468] time: 334.5015, train_loss: 0.02275885, train_accuracy: 1.0000, test_Accuracy: 0.9655
Epoch: [ 0] [  373/  468] time: 335.6037, train_loss: 0.06139914, train_accuracy: 0.9844, test_Accuracy: 0.9650
Epoch: [ 0] [  374/  468] time: 336.8191, train_loss: 0.06838398, train_accuracy: 0.9766, test_Accuracy: 0.9644
Epoch: [ 0] [  375/  468] time: 338.0675, train_loss: 0.08785215, train_accuracy: 0.9844, test_Accuracy: 0.9637
Epoch: [ 0] [  376/  468] time: 338.8702, train_loss: 0.12147868, train_accuracy: 0.9688, test_Accuracy: 0.9629
Epoch: [ 0] [  377/  468] time: 340.0370, train_loss: 0.07066518, train_accuracy: 0.9609, test_Accuracy: 0.9627
Epoch: [ 0] [  378/  468] time: 341.3515, train_loss: 0.23655194, train_accuracy: 0.9531, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 401.8138, train_loss: 0.04382807, train_accuracy: 0.9844, test_Accuracy: 0.9607
Epoch: [ 0] [  445/  468] time: 402.7321, train_loss: 0.06920724, train_accuracy: 0.9766, test_Accuracy: 0.9601
Epoch: [ 0] [  446/  468] time: 403.6230, train_loss: 0.10941057, train_accuracy: 0.9531, test_Accuracy: 0.9606
Epoch: [ 0] [  447/  468] time: 404.3539, train_loss: 0.10660991, train_accuracy: 0.9609, test_Accuracy: 0.9611
Epoch: [ 0] [  448/  468] time: 405.3487, train_loss: 0.02192589, train_accuracy: 0.9922, test_Accuracy: 0.9612
Epoch: [ 0] [  449/  468] time: 406.2779, train_loss: 0.08443705, train_accuracy: 0.9609, test_Accuracy: 0.9599
Epoch: [ 0] [  450/  468] time: 407.1547, train_loss: 0.10407702, train_accuracy: 0.9688, test_Accuracy: 0.9599
Epoch: [ 0] [  451/  468] time: 408.0933, train_loss: 0.08952449, train_accuracy: 0.9609, test_Accuracy: 0.9606
Epoch: [ 0] [  452/  468] time: 408.8970, train_loss: 0.06696184, train_accuracy: 0.9531, test_Accuracy:

# Test accuracy: 96.83%